<a href="https://colab.research.google.com/github/superizqi/BelajarBootstrap/blob/master/2_Pre_Data_Binning_(After_data_integration).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import io

from sklearn import preprocessing

#Uploading file
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive
from google.colab import files
uploaded = files.upload()

Saving [Hasil Integration] Ads_Performance_Instan Noodle_Food (All).xlsx to [Hasil Integration] Ads_Performance_Instan Noodle_Food (All) (1).xlsx


In [0]:
# Baca File 
namafile = '[Hasil Integration] Ads_Performance_Instan Noodle_Food (All).xlsx'
datall = pd.read_excel(namafile)
datall = datall.drop(['Unnamed: 0','No'],axis=1)
#Lihat data
datall.head()

,Date,Channel,Program,Product,Ads Type,Start Time,Duration,Cost,TVR
0,2019-01-10 00:00:00,ANTV,ISHQ MEIN MARJAWAN,POP MIE - INSTANT NOODLE,TEMPLATE,13:12:48,00:00:11,36000.0,1.70
1,2019-01-10 00:00:00,ANTV,ISHQ MEIN MARJAWAN,POP MIE - INSTANT NOODLE,VIRTUAL ADS,13:12:49,00:00:08,25159.0,1.70
2,2019-01-10 00:00:00,MNCTV,UPIN & IPIN BERMULA,SUPER BUBUR - INSTANT PORRIDGE,SQUEEZE FRAME,18:38:41,00:00:24,63000.0,1.70
3,2019-01-10 00:00:00,MNCTV,FATIH DI KAMPUNG JAWARA 2,MIGELAS JUMBO - INSTANT NOODLE,BUILT IN SEGMEN,20:14:43,00:02:30,325000.0,1.15
4,2019-01-10 00:00:00,RCTI,OGAH RUGI,POP MIE - INSTANT NOODLE,VIRTUAL ADS,15:14:21,00:00:15,48000.0,1.40


In [0]:
# Start method support for data binning
def equifreq(arr1, m): 
    # Binning by Frequency/Means
    a = len(arr1)
    arr1 = np.sort(arr1) 
    n = int(a / m) 
    databin = []
    for i in range(0, m): 
        arr = [] 
        for j in range(i * n, (i + 1) * n): 
            if j >= a: 
                break
            arr = arr + [arr1[j]] 
        # print(arr) 
        print(f'Total {len(arr)}')
        print(f'Sum of all the elements is {np.sum(arr)}')
        print("mean of arr : ", np.mean(arr))

def firstcheck(datacost):
  print("Cek first condition")
  cek = 0
  cektotal = 0
  datacost = np.sort(datacost)
  for i in range(0,3703,1):
    cek = cek + datacost[i]
    cektotal = cektotal + 1
  print("Frequency data = " + str(cektotal))
  print("Sum of data = " + str(cek))
  print("Mean of data = " + str(cek/3703))

def readdata(namakolom):
  datanya = []
  for d in datall[namakolom]:
    datanya.append(d)
  return datanya

# End method support for data binning

In [0]:
# Tambah TVR yang dibulatkan
tvr_round = []

for d in datall['TVR']:
  tvr_round.append(int(round(d)))
  
datall['TVR (One digit)'] = tvr_round
datall.head()

,Date,Channel,Program,Product,Ads Type,Start Time,Duration,Cost,TVR,TVR (One digit)
0,2019-01-10 00:00:00,ANTV,ISHQ MEIN MARJAWAN,POP MIE - INSTANT NOODLE,TEMPLATE,13:12:48,00:00:11,36000.0,1.70,2
1,2019-01-10 00:00:00,ANTV,ISHQ MEIN MARJAWAN,POP MIE - INSTANT NOODLE,VIRTUAL ADS,13:12:49,00:00:08,25159.0,1.70,2
2,2019-01-10 00:00:00,MNCTV,UPIN & IPIN BERMULA,SUPER BUBUR - INSTANT PORRIDGE,SQUEEZE FRAME,18:38:41,00:00:24,63000.0,1.70,2
3,2019-01-10 00:00:00,MNCTV,FATIH DI KAMPUNG JAWARA 2,MIGELAS JUMBO - INSTANT NOODLE,BUILT IN SEGMEN,20:14:43,00:02:30,325000.0,1.15,1
4,2019-01-10 00:00:00,RCTI,OGAH RUGI,POP MIE - INSTANT NOODLE,VIRTUAL ADS,15:14:21,00:00:15,48000.0,1.40,1


In [0]:
# datall['TVR (One digit)'].value_counts(ascending=True, dropna=False)
datall['TVR (One digit)'].value_counts(sort=False)

0     2847
1     6123
2     7803
3     2941
4     1845
5      437
6      131
7       14
8        1
9        5
10       7
11       5
12       3
Name: TVR (One digit), dtype: int64

In [0]:
# tutorial # https://towardsdatascience.com/getting-more-value-from-the-pandas-value-counts-aa17230907a6
# Binning by value counts function from pandas
datall['TVR (One digit)'].value_counts(bins=6)

(-0.013000000000000001, 2.0]    16794
(2.0, 4.0]                       4816
(4.0, 6.0]                        573
(6.0, 8.0]                         15
(8.0, 10.0]                        12
(10.0, 12.0]                        8
Name: TVR (One digit), dtype: int64

In [0]:
# binning manual
equifreq(readdata('TVR (One digit)'),6)
firstcheck(readdata('TVR (One digit)'))
# sepertinya kurang terbaca

Total 3703
Sum of all the elements is 855
mean of arr :  0.2308938698352687
Total 3703
Sum of all the elements is 3703
mean of arr :  1.0
Total 3703
Sum of all the elements is 5837
mean of arr :  1.5762894950040507
Total 3703
Sum of all the elements is 7406
mean of arr :  2.0
Total 3703
Sum of all the elements is 9127
mean of arr :  2.4647583040777747
Total 3703
Sum of all the elements is 14451
mean of arr :  3.9025114771806644
Cek first condition
Frequency data = 3703
Sum of data = 855
Mean of data = 0.2308938698352687


In [0]:
# Akhirnya saya bagi menjadi 6 Bin, dengan pembagian 0-2,3-4,5-6,7-8,9-10,diatas 10
databintvr = pd.cut(datall['TVR (One digit)'], bins=[0,2,4,6,8,10,12], labels=[int(1),int(2),int(3),int(4),int(5),int(6)])
if  'TVR Group' in datall.columns:
  datall = datall.drop('TVR Group',1)

datall['TVR Group'] = databintvr
datall.head()
# datall['TVR Group'].head()

,Date,Channel,Program,Product,Ads Type,Start Time,Duration,Cost,TVR,TVR (One digit),TVR Group
0,2019-01-10 00:00:00,ANTV,ISHQ MEIN MARJAWAN,POP MIE - INSTANT NOODLE,TEMPLATE,13:12:48,00:00:11,36000.0,1.70,2,1
1,2019-01-10 00:00:00,ANTV,ISHQ MEIN MARJAWAN,POP MIE - INSTANT NOODLE,VIRTUAL ADS,13:12:49,00:00:08,25159.0,1.70,2,1
2,2019-01-10 00:00:00,MNCTV,UPIN & IPIN BERMULA,SUPER BUBUR - INSTANT PORRIDGE,SQUEEZE FRAME,18:38:41,00:00:24,63000.0,1.70,2,1
3,2019-01-10 00:00:00,MNCTV,FATIH DI KAMPUNG JAWARA 2,MIGELAS JUMBO - INSTANT NOODLE,BUILT IN SEGMEN,20:14:43,00:02:30,325000.0,1.15,1,1
4,2019-01-10 00:00:00,RCTI,OGAH RUGI,POP MIE - INSTANT NOODLE,VIRTUAL ADS,15:14:21,00:00:15,48000.0,1.40,1,1


In [0]:
datall['TVR Group'].value_counts()
# datall.hist(column="TVR (One digit)")

1    13946
2     4816
3      573
4       15
5       12
6        8
Name: TVR Group, dtype: int64

In [0]:
# Start data binning untuk kolom cost
# tutorial # https://towardsdatascience.com/getting-more-value-from-the-pandas-value-counts-aa17230907a6
# Binning by value counts function from pandas
datall['Cost'].value_counts(bins=6)

(-2999.9010000000003, 500000.083]    22162
(500000.083, 1000000.067]               46
(1000000.067, 1500000.05]                5
(2000000.033, 2500000.017]               2
(1500000.05, 2000000.033]                2
(2500000.017, 3000000.0]                 1
Name: Cost, dtype: int64

In [0]:
# menghapus data pencilan

if len(datall['Cost']) > 22162:
  totaldata = 0 
  for i in datall['Cost']:
    # cek totalnya
    if int(i) > 500000.083:
      totaldata = totaldata + 1
      # datall['Cost'].drop([i, inplace = True) 
  print(totaldata)
  # print(len(datall['Cost']))
  # ambil data
  indexNames = datall[datall['Cost'] > 500000 ].index
  # hapus data
  datall.drop(indexNames , inplace=True)

print("Before = " + str(len(datall['Cost'])+56))
print("After = " + str(len(datall['Cost'])))

Before = 22218
After = 22162


In [0]:
# Start data binning setelah direduksi untuk kolom cost
# tutorial # https://towardsdatascience.com/getting-more-value-from-the-pandas-value-counts-aa17230907a6
# Binning by value counts function from pandas
datall['Cost'].value_counts(bins=6, sort=False)

(-499.90099999999995, 83333.417]    21761
(83333.417, 166666.733]               231
(166666.733, 250000.05]                13
(250000.05, 333333.367]                37
(333333.367, 416666.683]               34
(416666.683, 500000.0]                 86
Name: Cost, dtype: int64

In [0]:
# binning manual
equifreq(readdata('Cost'),6)
firstcheck(readdata('Cost'))

Total 3693
Sum of all the elements is 47933587.64
mean of arr :  12979.57964798267
Total 3693
Sum of all the elements is 78765793.8
mean of arr :  21328.403411860276
Total 3693
Sum of all the elements is 106400275.8
mean of arr :  28811.339236393174
Total 3693
Sum of all the elements is 142815172.2
mean of arr :  38671.85816409423
Total 3693
Sum of all the elements is 184616498.89999998
mean of arr :  49990.92848632547
Total 3693
Sum of all the elements is 300870824.0
mean of arr :  81470.57243433523
Cek first condition
Frequency data = 3703
Sum of data = 48113587.63999997
Mean of data = 12993.137358898182


In [0]:
# Akhirnya saya bagi menjadi 6 Bin sesuai dengan yang dibagi secara default dari pandas karena lebih mudah dipahami
# yaitu 0 - 80000, 80001 - 160000, 160001-240000, 240001 - 320000, 320001 - 400000, diatas 400000 
databincost = pd.cut(datall['Cost'], bins=[0,80000,160000,240000,320000,400000,600000], labels=[int(1),int(2),int(3),int(4),int(5),int(6)])
if  'Cost Group' in datall.columns:
  datall = datall.drop('Cost Group',1)

datall['Cost Group'] = databincost
# datall.head()
print(len(datall))

22162


In [0]:
datall['Cost Group'].value_counts(sort=False)
# datall.hist(column="Cost Group")

1    21760
2      232
3       12
4       35
5       37
6       86
Name: Cost Group, dtype: int64

In [0]:
# datall['Cost Group'].value_counts(normalize=True)

In [0]:
def databinning(namakolom, jumlahbin):
  datanya = []

  for d in datall[namakolom]:
      datanya.append(d)
      # arr_tvr.append(int(round(d)))

  print(len(datanya))

  datanya = np.sort(datanya)
  # datanya = datanya.sort()
  print(datanya)
  de = 0

  for i in range(0,len(datanya),int(len(datanya)/jumlahbin)):
    mean = 0  
    for j in range (0,int(len(datanya)/jumlahbin),1) :
      mean = mean + datanya[i+j]
    de = de + 1
    print("Bin " + str(de) + " Batas bawah " + str(i) + " Batas atas " + str(i+(datanya.size/jumlahbin)) + " Mean " + str(mean/((datanya.size/jumlahbin))))

databinning('Cost',6)
print("Nilai paling tinggi = " + str(max(datall['Cost'])))

22218
[1.0e-01 1.0e-01 1.0e-01 ... 2.2e+06 2.5e+06 3.0e+06]
Bin 1 Batas bawah 0 Batas atas 3703.0 Mean 12993.137358898182
Bin 2 Batas bawah 3703 Batas atas 7406.0 Mean 21368.02425060757
Bin 3 Batas bawah 7406 Batas atas 11109.0 Mean 28855.056926816233
Bin 4 Batas bawah 11109 Batas atas 14812.0 Mean 38753.759708344565
Bin 5 Batas bawah 14812 Batas atas 18515.0 Mean 50212.39505806108
Bin 6 Batas bawah 18515 Batas atas 22218.0 Mean 93601.35673778021
Nilai paling tinggi = 3000000.0


In [0]:
databincost = pd.cut(datall['Cost'], bins=[0,5000,10000,15000,20000,25000,30000,35000,40000,45000,300000],
                     labels=[int(1),int(2),int(3),int(4),int(5),int(6),int(7),int(8),int(9),int(10)])
if  'Cost Group' in datall.columns:
  datall = datall.drop('Cost Group',1)

datall.insert(5,'Cost Group',databincost)
datall['Cost Group'].value_counts(sort=False)

1      245
2      375
3     1813
4     3252
5      974
6     3469
7     1382
8     2221
9     1472
10    6832
Name: Cost Group, dtype: int64

In [0]:
# datall['Cost Group'].value_counts()
datall.head()

,Date,Channel,Program,Product,Ads Type,Start Time,Duration,Cost,TVR,TVR (Round)
0,2019-01-10 00:00:00,ANTV,ISHQ MEIN MARJAWAN,POP MIE - INSTANT NOODLE,TEMPLATE,13:12:48,00:00:11,36000.0,1.70,2
1,2019-01-10 00:00:00,ANTV,ISHQ MEIN MARJAWAN,POP MIE - INSTANT NOODLE,VIRTUAL ADS,13:12:49,00:00:08,25159.0,1.70,2
2,2019-01-10 00:00:00,MNCTV,UPIN & IPIN BERMULA,SUPER BUBUR - INSTANT PORRIDGE,SQUEEZE FRAME,18:38:41,00:00:24,63000.0,1.70,2
3,2019-01-10 00:00:00,MNCTV,FATIH DI KAMPUNG JAWARA 2,MIGELAS JUMBO - INSTANT NOODLE,BUILT IN SEGMEN,20:14:43,00:02:30,325000.0,1.15,1
4,2019-01-10 00:00:00,RCTI,OGAH RUGI,POP MIE - INSTANT NOODLE,VIRTUAL ADS,15:14:21,00:00:15,48000.0,1.40,1


In [0]:
def equifreq(arr1, m): 
    # Binning by Frequency/Means
    a = len(arr1)
    arr1 = np.sort(arr1) 
    n = int(a / m) 
    databin = []
    for i in range(0, m): 
        arr = [] 
        for j in range(i * n, (i + 1) * n): 
            if j >= a: 
                break
            arr = arr + [arr1[j]] 
        # print(arr) 
        print(f'Total {len(arr)}')
        print(f'Sum of all the elements is {np.sum(arr)}')
        print("mean of arr : ", np.mean(arr))

def firstcheck(datacost):
  print("Cek first condition")
  cek = 0
  cektotal = 0
  datacost = np.sort(datacost)
  for i in range(0,3703,1):
    cek = cek + datacost[i]
    cektotal = cektotal + 1
  print("Frequency data = " + str(cektotal))
  print("Sum of data = " + str(cek))
  print("Mean of data = " + str(cek/3703))

def readdata(namakolom):
  datanya = []
  for d in datall[namakolom]:
    datanya.append(d)
  return datanya

In [0]:
equifreq(readdata('Cost'), 6) 
firstcheck(readdata('Cost'))

Total 3703
Sum of all the elements is 48113587.64
mean of arr :  12993.137358898191
Total 3703
Sum of all the elements is 79125793.8
mean of arr :  21368.024250607614
Total 3703
Sum of all the elements is 106850275.80000001
mean of arr :  28855.056926816098
Total 3703
Sum of all the elements is 143505172.2
mean of arr :  38753.75970834458
Total 3703
Sum of all the elements is 185936498.89999998
mean of arr :  50212.395058061025
Total 3703
Sum of all the elements is 346605824.0
mean of arr :  93601.35673778018
Cek first condition
Frequency data = 3703
Sum of data = 48113587.63999997
Mean of data = 12993.137358898182


In [0]:
equifreq(readdata('TVR'), 6) 
firstcheck(readdata('TVR'))

Total 3703
Sum of all the elements is 1206.65
mean of arr :  0.3258574129084526
Total 3703
Sum of all the elements is 3961.6500000000005
mean of arr :  1.0698487712665408
Total 3703
Sum of all the elements is 5490.2
mean of arr :  1.4826357007831488
Total 3703
Sum of all the elements is 7019.25
mean of arr :  1.8955576559546314
Total 3703
Sum of all the elements is 9381.45
mean of arr :  2.5334728598433705
Total 3703
Sum of all the elements is 14498.75
mean of arr :  3.915406427221172
Cek first condition
Frequency data = 3703
Sum of data = 1206.6500000000044
Mean of data = 0.32585741290845377


In [0]:
equifreq(readdata('TVR (Round)'), 6) 
firstcheck(readdata('TVR (Round)'))

Total 3703
Sum of all the elements is 855
mean of arr :  0.0
Total 3703
Sum of all the elements is 3703
mean of arr :  1.0
Total 3703
Sum of all the elements is 5837
mean of arr :  2.0
Total 3703
Sum of all the elements is 7406
mean of arr :  2.0
Total 3703
Sum of all the elements is 9127
mean of arr :  2.0
Total 3703
Sum of all the elements is 14451
mean of arr :  4.0
Cek first condition
Frequency data = 3703
Sum of data = 855
Mean of data = 0.0


In [0]:
# Cek Algoritma sesuai tutorial
# https://www.geeksforgeeks.org/python-binning-method-for-data-smoothing/
equifreq([4, 8, 9, 15, 21, 21, 24, 25, 26, 28, 29, 34],3)

Total 4
Sum of all the elements is 36
mean of arr :  9.0
Total 4
Sum of all the elements is 91
mean of arr :  23.0
Total 4
Sum of all the elements is 117
mean of arr :  29.0
